I'm using this sheet to help me create various technical metrics that the agent can use to determine if a company is under or over valued.

In [1]:
% pip install -U openbb

UsageError: Line magic function `%` not found.


In [2]:
from openbb import obb
from datetime import datetime, timedelta
import os

In [3]:
obb.account.login(pat=os.environ.get("OPENBB_PAT"))
obb.user.credentials

Credentials

benzinga_api_key: None
fmp_api_key: **********
fred_api_key: **********
intrinio_api_key: None
polygon_api_key: **********
tiingo_token: None
tradingeconomics_api_key: **********

In [4]:
now = datetime.now()
end_date = datetime(
    year=now.year,
    month=now.month,
    day=now.day)
delta = timedelta(days=30)
start_date = end_date - delta


output = obb.equity.price.historical("AAPL", start_date=start_date, end_date=end_date, provider='yfinance')
df = output.to_dataframe()
df.head()
df

,open,high,low,close,volume
date,,,,,
2024-12-13,247.820007,249.289993,246.240005,248.130005,33155300
2024-12-16,247.990005,251.380005,247.649994,251.039993,51694800
2024-12-17,250.080002,253.830002,249.779999,253.479996,51356400
2024-12-18,252.160004,254.279999,247.740005,248.050003,56774100
2024-12-19,247.500000,252.000000,247.089996,249.789993,60882300
2024-12-20,248.039993,255.000000,245.690002,254.490005,147495300
2024-12-23,254.770004,255.649994,253.449997,255.270004,40858800
2024-12-24,255.490005,258.209991,255.289993,258.200012,23234700
2024-12-26,258.190002,260.100006,257.630005,259.019989,27237100


In [ ]:
obb.equity.search("JPMorgan", provider="sec").to_df().head(3)

In [ ]:
obb.etf.search("gold", provider="fmp").to_df().iloc[-5:]

In [5]:
all_companies = obb.equity.search("", provider="sec")
print(len(all_companies.results))
all_companies.to_df().head(10)

10015


,symbol,name,cik
0,AAPL,Apple Inc.,320193
1,NVDA,NVIDIA CORP,1045810
2,MSFT,MICROSOFT CORP,789019
3,GOOGL,Alphabet Inc.,1652044
4,AMZN,AMAZON COM INC,1018724
5,META,"Meta Platforms, Inc.",1326801
6,TSLA,"Tesla, Inc.",1318605
7,AVGO,Broadcom Inc.,1730168
8,BRK-B,BERKSHIRE HATHAWAY INC,1067983
9,WMT,Walmart Inc.,104169


In [47]:
results = obb.equity.screener(# , sector="technology"
                              # , industry="semiconductors"
                              country="US"
                              , exchange="nasdaq"
                              , provider="fmp" # order "yfinance"
                              ).to_df()
                              
results = results[results.is_etf == False] \
                 [results.isFund == False] \
                 [results.actively_trading == True] \
                 [results.price > 1] \
                 [results.market_cap > 300000000] \
                 [results.volume > 10000]


display(f"Results: {len(results)}")
# display(f"Exchanges: {len(exchanges)}")
results.convert_dtypes()

/tmp/ipykernel_67011/2901545030.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  results = results[results.is_etf == False] \
/tmp/ipykernel_67011/2901545030.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  results = results[results.is_etf == False] \
/tmp/ipykernel_67011/2901545030.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  results = results[results.is_etf == False] \
/tmp/ipykernel_67011/2901545030.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  results = results[results.is_etf == False] \


'Results: 1213'

TypeError: object of type 'method' has no len()

In [62]:
exchanges = results['exchange'].unique()
print(exchanges) #.tolist())

['NASDAQ']


In [7]:
results.columns

Index(['symbol', 'name', 'market_cap', 'sector', 'industry', 'beta', 'price',
       'last_annual_dividend', 'volume', 'exchange', 'exchange_name',
       'country', 'is_etf', 'actively_trading', 'isFund'],
      dtype='object')

In [8]:
results["dividend_yield"] = results["last_annual_dividend"]/results["price"]
results.head()

,symbol,name,market_cap,sector,industry,beta,price,last_annual_dividend,volume,exchange,exchange_name,country,is_etf,actively_trading,isFund,dividend_yield
0,AAPL,Apple Inc.,3.580177e+12,Technology,Consumer Electronics,1.240,236.85,0.99,58293813.0,NASDAQ,NASDAQ Global Select,US,False,True,False,0.004180
1,NVDA,NVIDIA Corporation,3.328436e+12,Technology,Semiconductors,1.657,135.91,0.04,204822623.0,NASDAQ,NASDAQ Global Select,US,False,True,False,0.000294
2,MSFT,Microsoft Corporation,3.114843e+12,Technology,Software - Infrastructure,0.904,418.95,3.08,19826981.0,NASDAQ,NASDAQ Global Select,US,False,True,False,0.007352
3,GOOG,Alphabet Inc.,2.357015e+12,Communication Services,Internet Content & Information,1.034,193.17,0.60,18566759.0,NASDAQ,NASDAQ Global Select,US,False,True,False,0.003106
4,GOOGL,Alphabet Inc.,2.357015e+12,Communication Services,Internet Content & Information,1.034,192.04,0.60,25712962.0,NASDAQ,NASDAQ Global Select,US,False,True,False,0.003124


In [9]:
security = "TGT"

In [11]:
# EPS = Net Income - Preferred Dividends / Weighted Average Common Shares

appl_fundamentals = obb.equity.fundamental.balance(security, provider="fmp", limit=3)
appl_fundamentals.results[0]

FMPBalanceSheetData(period_ending=2024-02-03, fiscal_period=FY, fiscal_year=2023, filing_date=2024-03-13, accepted_date=2024-03-13 17:04:51, reported_currency=USD, cash_and_cash_equivalents=3805000000.0, short_term_investments=None, cash_and_short_term_investments=3805000000.0, net_receivables=1404000000.0, inventory=11886000000.0, other_current_assets=403000000.0, total_current_assets=17498000000.0, plant_property_equipment_net=36458000000.0, goodwill=631000000.0, intangible_assets=8000000.0, goodwill_and_intangible_assets=639000000.0, long_term_investments=None, tax_assets=None, other_non_current_assets=761000000.0, non_current_assets=37858000000.0, other_assets=None, total_assets=55356000000.0, accounts_payable=12098000000.0, short_term_debt=1445000000.0, tax_payables=827000000.0, current_deferred_revenue=None, other_current_liabilities=4934000000.0, total_current_liabilities=19304000000.0, long_term_debt=16307000000.0, deferred_revenue_non_current=419000000.0, deferred_tax_liabilit

In [25]:
fundamental_data = obb.equity.fundamental.income( security, provider="yfinance", limit=1, period="quarter" ).to_df()
fundamental_data

,period_ending,operating_revenue,total_revenue,cost_of_revenue,gross_profit,selling_general_and_admin_expense,depreciation_and_amortization_in_income_statement,depreciation_amortization_depletion_income_statement,operating_expense,operating_income,...,interest_expense,net_interest_income,ebit,ebitda,reconciled_cost_of_revenue,reconciled_depreciation,net_income_from_continuing_operation_net_minority_interest,normalized_ebitda,tax_rate_for_calcs,tax_effect_of_unusual_items
0,2024-10-31,2.522800e+10,2.566800e+10,1.837500e+10,7.293000e+09,5.486000e+09,639000000.0,639000000.0,6.125000e+09,1.168000e+09,...,105000000.0,-105000000.0,1.196000e+09,1.950000e+09,1.826000e+10,754000000.0,854000000.0,1.950000e+09,0.217,0.0


In [27]:
results["total_revenue"] = obb.equity.fundamental.balance(results['symbol'], provider="yfinance", limit=1, period="quarter").to_df()['total_revenue']

OpenBBError: 
[Error] -> 1 validations error(s)
[Arg] 1 -> input: 0        AAPL
1        NVDA
2        MSFT
3        GOOG
4       GOOGL
        ...  
4149    FMOXX
4150    FLGXX
4151    FDRXX
4152    FDLXX
4153    BFRGW
Name: symbol, Length: 4154, dtype: object -> Input should be a valid string

In [21]:
# Weighted Average Shares Outstanding
data = obb.equity.fundamental.income( security, provider="yfinance", limit=5, period="quarter" ).to_df()
shares = data["weighted_average_basic_shares_outstanding"]
results["eps"] = obb.equity.fundamental.income(results['symbol'], provider="yfinance", limit=5, period="quarter").to_df()['weighted_average_basic_shares_outstanding'][0] / obb.equity.fundamental.income(results['symbol'], provider="yfinance", limit=5, period="quarter")['weighted_average_basic_shares_outstanding']
display(shares.head(1))

display(shares.tail(1))

KeyError: 0